In [1]:
import requests 
import json 
import pandas as pd
import config 

#connection
bearer_token='AAAAAAAAAAAAAAAAAAAAAEZsigEAAAAA6sTnwgKifd9%2FfmRkhlDObjJP8ao%3DadxQiV9Ou59TOEu4bTbiJ7xwXTs6GGnCaMqGGH6nhY0hZgjBHD'
# from pytwitter import Api
# api = Api(bearer_token=bearer_token)

# from pytwitter import StreamApi
# stream_api = StreamApi(bearer_token=bearer_token)

https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/main/Filtered-Stream/filtered_stream.py

In [2]:

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FilteredStreamPython"
    return r



In [3]:
def get_rules():
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    return response.json()



In [4]:
def delete_all_rules(rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print(json.dumps(response.json()))


In [5]:
search_rules = [
   # {"tag": 'AI OR CVE OR Cyberspace OR DoS Attack '},
    {"value": "CVE OR Cyberspace OR DoS Attack OR Exploit OR Hacker", "tag": "CVE OR Cyberspace OR DoS Attack OR Exploit OR Hacker "},
    {"value": "malware OR Vulnerability OR cybersecurity", "tag": "malware OR Vulnerability OR cybersecurity"},
    {"value": "IT Security OR Malware OR MTTR OR Patch OR Penetration Testing", "tag": "IT Security OR Malware OR MTTR OR Patch OR Penetration Testing "},
    {"value": " Phishing OR Ransomware OR Red Team OR Risk OR Threat OR Security programs", "tag": " Phishing OR Ransomware OR Red Team OR Risk OR Threat OR Security programs "},
]

In [6]:
def set_rules(delete):
    # You can adjust the rules if needed
    sample_rules = [
        {"value": "CVE OR Cyberspace OR DoS Attack OR Exploit OR Hacker", "tag": "CVE OR Cyberspace OR DoS Attack OR Exploit OR Hacker "},
        {"value": "malware OR Vulnerability OR cybersecurity", "tag": "malware OR Vulnerability OR cybersecurity"},
    ]
    payload = {"add": search_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))


In [7]:
tweet_list = []

# appends information from tweets to a dataframe                
def save_tweets(tweet):

    data = tweet['data']
    public_metrics = data['public_metrics']

    id = data['id']
    author_id = data['author_id']
    full_text = data['text']
    lang = data['lang']
    created_at = data['created_at']

    retweets = public_metrics['retweet_count']
    replies = public_metrics['reply_count']
    likes = public_metrics['like_count']
    quotes =  public_metrics['quote_count']

    
    tweet_list.append([id,full_text, author_id,  lang, created_at,
                        retweets, replies, likes, quotes])

    

In [8]:
fields = "?tweet.fields=attachments,author_id,created_at,public_metrics"
expansions = " "
def get_stream(set):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream" + expansions + fields, auth=bearer_oauth,  
        stream=True, 
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            save_tweets(json_response)
            print(json.dumps(json_response, indent=4, sort_keys=True))




In [9]:
def main():
    rules = get_rules()
    delete = delete_all_rules(rules)
    set = set_rules(delete)
    get_stream(set)



In [10]:

main()


{"data": [{"id": "1651559023005560834", "value": "malware"}], "meta": {"sent": "2023-05-27T16:30:03.330Z", "result_count": 1}}
{"meta": {"sent": "2023-05-27T16:30:04.642Z", "summary": {"deleted": 1, "not_deleted": 0}}}
{"data": [{"value": "malware OR Vulnerability OR cybersecurity", "tag": "malware OR Vulnerability OR cybersecurity", "id": "1662496403745112067"}, {"value": " Phishing OR Ransomware OR Red Team OR Risk OR Threat OR Security programs", "tag": " Phishing OR Ransomware OR Red Team OR Risk OR Threat OR Security programs ", "id": "1662496403745112065"}, {"value": "CVE OR Cyberspace OR DoS Attack OR Exploit OR Hacker", "tag": "CVE OR Cyberspace OR DoS Attack OR Exploit OR Hacker ", "id": "1662496403745112064"}, {"value": "IT Security OR Malware OR MTTR OR Patch OR Penetration Testing", "tag": "IT Security OR Malware OR MTTR OR Patch OR Penetration Testing ", "id": "1662496403745112066"}], "meta": {"sent": "2023-05-27T16:30:06.439Z", "summary": {"created": 4, "not_created": 0, 

Exception: Cannot get stream (HTTP 404): 

In [11]:
streamed = pd.DataFrame(tweet_list, columns = ['tweet_id', 'full_text', 'author_id', 'lang', 'created_at',
                        'retweets', 'replies', 'likes', 'quotes'])


In [12]:
streamed['full_text'][0]

IndexError: index 0 is out of bounds for axis 0 with size 0

unsuccessfull part here:


In [13]:
import time

In [14]:
keyword_dict = [{"value": 'sql injection OR code execution OR remote code OR privilege escalation OR buffer overflow OR arbitrary code'},
                {"value": 'injection vulnerability OR command injection OR execute arbitrary OR remote attacker OR execution vulnerability'},
                {"value": 'remote attackers OR allows remote OR critical cve OR medium https OR cve detected OR out-of-bounds write'},
                {"value": 'cross-site scripting OR privilege vulnerability OR escalation vulnerability OR write vulnerability OR patches critical'},
                {"value": 'injection attack OR use-after-free vulnerability OR execution vulnerabilities OR injection vulnerabilities'},
                {"value": 'integer overflow OR code injection OR execution privileges OR authenticated attacker OR injection attacks'},
                {"value": 'malicious code OR remote code execution OR sql injection type OR code execution vulnerability OR sql injection vulnerability'},
                {"value": 'execute arbitrary code OR arbitrary code execution OR potentially critical cve OR critical cve detected'},
                {"value": 'allows remote attackers OR local privilege escalation OR vulnerability allows attackers OR buffer overflow vulnerability'},
                {"value": 'heap-based buffer overflow OR heap buffer overflow OR privilege escalation vulnerability OR vulnerability allows remote'},
                {"value": 'arbitrary code via OR exploit heap corruption OR stack-based buffer overflow OR injection vulnerability exists'},
                {"value": 'out-of-bounds write vulnerability OR sql injection attack OR unauthenticated remote attacker OR code execution vulnerabilities'},
                {"value": 'overflow vulnerability exists OR privilege escalation in OR specially-crafted http request OR sql injection attacks'},
                {"value": 'unauthenticated remote code OR stack overflow vulnerability OR allow remote attackers OR crafted payload injected'},
                {"value": 'out-of-bounds read OR command injection vulnerability OR cross-site request forgery OR overflow vulnerability OR zero-day OR 0-day'},
                {"value": 'malware OR ransomware OR spam'}]

In [15]:
second_stub_dict = [{"value": 'sql injection OR code execution OR remote code OR privilege escalation OR buffer overflow OR arbitrary code'},
                    {"value": 'injection vulnerability OR command injection OR execute arbitrary OR remote attacker OR execution vulnerability'},
                    {"value": 'remote attackers OR allows remote OR critical cve OR medium https OR cve detected OR out-of-bounds write'},
                    {"value": 'cross-site scripting OR privilege vulnerability OR escalation vulnerability OR write vulnerability OR patches critical'},
                    {"value": 'injection attack OR use-after-free vulnerability OR execution vulnerabilities OR injection vulnerabilities'},
                    {"value": 'integer overflow OR code injection OR execution privileges OR authenticated attacker OR injection attacks'},
                    {"value": 'malicious code OR remote code execution OR sql injection type OR code execution vulnerability OR sql injection vulnerability'},
                    {"value": 'execute arbitrary code OR arbitrary code execution OR potentially critical cve OR critical cve detected'},
                    {"value": 'allows remote attackers OR local privilege escalation OR vulnerability allows attackers OR buffer overflow vulnerability'},
                    {"value": 'heap-based buffer overflow OR heap buffer overflow OR privilege escalation vulnerability OR vulnerability allows remote'},
                    {"value": 'arbitrary code via OR exploit heap corruption OR stack-based buffer overflow OR injection vulnerability exists'},
                    {"value": 'out-of-bounds write vulnerability OR sql injection attack OR unauthenticated remote attacker OR code execution vulnerabilities'},
                    {"value": 'overflow vulnerability exists OR privilege escalation in OR specially-crafted http request OR sql injection attacks'},
                    {"value": 'unauthenticated remote code OR stack overflow vulnerability OR allow remote attackers OR crafted payload injected'},
                    {"value": 'out-of-bounds read OR command injection vulnerability OR cross-site request forgery OR overflow vulnerability OR zero-day OR 0-day'}]

In [17]:
# the max number of tweets that will be returned at a time
max_results = 5000


# we'll use the following keywords
# search_rules = second_stub_dict
search_rules = second_stub_dict

def get_params():
    return {"tweet.fields": ["entities,public_metrics,lang,created_at"], "user.fields": ["location"],
             "expansions":"author_id,geo.place_id", "place.fields" : ['country']
}

params = get_params()

In [18]:
# puts the bearer token in the request header
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [19]:
def set_rules(headers, delete, bearer_token, search_rules):
    search_rules = keyword_dict
    payload = {"add": search_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )


In [20]:
# retrieves the current set of rules from the API     
def get_rules(headers, bearer_token):
    response = requests.get(
        # "https://api.twitter.com/2/tweets/sample/stream",

        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    #print(json.dumps(response.json()))
    return response.json()

In [21]:
# tells the API to delete our current rule configuration 
def delete_all_rules(headers, bearer_token, rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        # "https://api.twitter.com/2/tweets/sample/stream",

        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    #print(json.dumps(response.json()))

In [22]:
# starts the stream, iterates through the lines of the response and for each line calls the save_tweets function
data = []
def get_stream(headers, set, bearer_token,  fields, max_results):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream" , 
        headers=headers, stream=True, params=params
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    i = 0
    for response_line in response.iter_lines():
        i += 1
        if i == max_results:
            break
        elif response_line:
            json_response = json.loads(response_line)
            data.append(json_response)
            save_tweets(json_response)
        else: 
            # In case the JSON fails to decode, we skip this tweet
            print(f"{i}/{max_results}: ERROR: encountered a problem with a line of data... \n")
        


In [23]:
tweet_list = []
# appends information from tweets to a dataframe                
def save_tweets(tweet):
    
    data = tweet['data']
    public_metrics = data['public_metrics']

    id = data['id']
    author_id = data['author_id']
    full_text = data['text']
    lang = data['lang']
    created_at = data['created_at']
    geo = data['geo']

    retweets = public_metrics['retweet_count']
    replies = public_metrics['reply_count']
    likes = public_metrics['like_count']
    quotes =  public_metrics['quote_count']

    includes = tweet['includes']
    if 'location' in includes['users'][0]:
        location = includes['users'][0]['location']
    else:
        location =""
    # user_creation = includes['users'][0]['created_at'] 
    # followers =  includes['users'][0]['public_metrics']['followers_count'] 
    # followings = includes['users'][0]['public_metrics']['following_count'] 
    # no_of_tweets = includes['users'][0]['public_metrics']['listed_count'] 
    # user_verified = includes['users'][0]['verified']
    # user_protected = includes['users'][0]['protected']
    
    tweet_list.append([id, author_id, full_text, lang, created_at, geo,
                        retweets, replies, likes, quotes, location])
                        # username, location, user_creation, followers,
                        # followings, no_of_tweets, user_verified, user_protected ])


In [24]:
bearer_token = bearer_token
headers = create_headers(bearer_token)
rules = get_rules(headers, bearer_token)
delete = delete_all_rules(headers, bearer_token, rules)
set = set_rules(headers, delete, bearer_token, search_rules)
get_stream(headers, set, bearer_token,  params, 2000)

columns = [ #tweet data
            'tweetid', 'author_id' ,'full_text', 'lang', 'created_at','geo',
            'retweets', 'replies', 'likes', 'quotes',  'location'
    
            # 'retweeted', 'favorited', 'is_quote_status', 'retweet_count', 
            # 'favorite_count', 'lang', 'in_reply_to_status_id', 'in_reply_to_user_id',
            # 'hashtags', 'user_mentions', 'urls'
            # public metrics
            #'like_count'
            #'quote_count', 'reply_count', 'retweet_count' ,
            # media
            # non public metrics
            #'url_link_clicks'
            ]



200
297/2000: ERROR: encountered a problem with a line of data... 

630/2000: ERROR: encountered a problem with a line of data... 

957/2000: ERROR: encountered a problem with a line of data... 

1370/2000: ERROR: encountered a problem with a line of data... 

1701/2000: ERROR: encountered a problem with a line of data... 



In [25]:

columns = [ #tweet data
            'tweetid', 'author_id' ,'full_text', 'lang', 'created_at','geo',
            'retweets', 'replies', 'likes', 'quotes',  'location'
    
            # 'retweeted', 'favorited', 'is_quote_status', 'retweet_count', 
            # 'favorite_count', 'lang', 'in_reply_to_status_id', 'in_reply_to_user_id',
            # 'hashtags', 'user_mentions', 'urls'
            # public metrics
            #'like_count'
            #'quote_count', 'reply_count', 'retweet_count' ,
            # media
            # non public metrics
            #'url_link_clicks'
            ]

df = pd.DataFrame (tweet_list, columns = columns)
df.explode('geo')
df


,tweetid,author_id,full_text,lang,created_at,geo,retweets,replies,likes,quotes,location
0,1662496630233350144,4901180902,@Blackkhally @Oluhill @UchennaEkechuk2 @SirJar...,en,2023-05-27T16:30:58.000Z,{},0,0,0,0,"Dallas, TX"
1,1662496631575437322,1381889142741737473,@DonkTrading Can anyone find a picture of a gu...,en,2023-05-27T16:30:59.000Z,{},0,0,0,0,
2,1662496632301146113,1520526732171829252,@DM3710Liberty @toni_triano @laralogan She had...,en,2023-05-27T16:30:59.000Z,{},0,0,0,0,🍌Banana 🍌Republic 🇺🇸
3,1662496630090657803,1522634406850088962,Right wing extremist at it again...just kiddin...,en,2023-05-27T16:30:58.000Z,{},0,0,0,0,Ont. Canada
4,1662496631399350274,179407588,RT @Doranimated: Erdogan took a principled pos...,en,2023-05-27T16:30:59.000Z,{},176,0,0,0,"Alanya, Turkiye"
...,...,...,...,...,...,...,...,...,...,...,...
1989,1662497105343295488,1328036804629458944,RT @FoxFairyProject: 🌐Website:https://t.co/tGN...,en,2023-05-27T16:32:52.000Z,{},499,0,0,0,"Pune, India"
1990,1662497102474399744,1496504146803171329,RT @kyokuyo_PR: ／\nフォロー＆RTでOK！\nとろける柔らかさにびっくり😳...,ja,2023-05-27T16:32:51.000Z,{},5978,0,0,0,
1991,1662497105175359489,1362436981,RT @RevistaSemana: #Política | Reforma a la sa...,es,2023-05-27T16:32:52.000Z,{},195,0,0,0,Pereira
1992,1662497105049526273,1330487908655112201,"INTERVIEW (May 18, 2023) - Man in America Podc...",en,2023-05-27T16:32:52.000Z,{},0,0,0,0,


In [26]:
for i in range(0, len(df)):
    if (df['geo'][i] != {}):
        print(df['geo'][i])

{'place_id': 'a4ddc3856053f7e1'}
{'place_id': '01fbe706f872cb32'}
{'place_id': '4efd6fc8c50fae33'}
{'place_id': 'cd450c94084cbf9b'}
{'place_id': '5ef5b7f391e30aff'}


In [27]:
df.columns

Index(['tweetid', 'author_id', 'full_text', 'lang', 'created_at', 'geo',
       'retweets', 'replies', 'likes', 'quotes', 'location'],
      dtype='object')

In [28]:
def data_save(data_to_u2v):
    data_to_u2v['tweet_id'] = data_to_u2v['tweetid'].astype(str)
    # data_to_u2v['conversation_id'] = data_to_u2v['conversation_id'].astype(str)
    data_to_u2v['author_id'] = data_to_u2v['author_id'].astype(str)
data_save(df)


In [29]:
df = df.drop_duplicates(subset=['full_text'])
df.drop_duplicates(subset=['tweetid'])
df.shape

(1642, 12)

In [30]:
df['full_text'][32]

'RT @moost964: kaitun07 น้องไข่ตุ๋น เปิดนมสวยๆ วัยรุ่นไทยน่ารัก | หนังโป๊เว้ยเฮ้ย.net https://t.co/CAHadNiJRH'

In [31]:
df.to_excel('malware4.xlsx', encoding = 'utf-8-sig')

C:\Users\Nastea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
